In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - track_cutout_downloads.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstring:
'''Look and see how many cutouts we have downloaded and how many we still need 
to download.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb
import h5py
import glob
import copy
import dill as pickle
from astropy import units as apu

## Matplotlib
import matplotlib
from matplotlib import pyplot as plt

## Galpy
from galpy import orbit, potential, df
import galpy.util

## Scipy
import scipy.interpolate

sys.path.insert(0,'../../src/')
from tng_dfs import util as putil
from tng_dfs import tree as ptree
from tng_dfs import cutout as pcutout
from tng_dfs.util import get

### Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','RO','VO','ZO','LITTLE_H']
data_dir,ro,vo,zo,h = putil.parse_config_dict(cdict,keywords)

### Functions

### API Setup & Milky Way Analogs from TNG50-1

In [ ]:
# Base URL
baseURL = 'http://www.tng-project.org/api/'
# Get list of simulations
r = get(baseURL)
sim_names = [sim['name'] for sim in r['simulations']]
tng50_indices = [sim_names.index('TNG50-'+str(i+1)) for i in range(4)]
# Choose the lowest resolution tng50 run
tng50_urls = [r['simulations'][i]['url'] for i in tng50_indices]
tng50_url = tng50_urls[0]

# Get the simulation, snapshots, snapshot redshifts
sim = get( tng50_url )
snaps = get( sim['snapshots'] )

In [ ]:
with open('../parse_sublink_trees/data/tree_primaries.pkl','rb') as handle:
    tree_primaries = pickle.load(handle)
with open('../parse_sublink_trees/data/tree_major_mergers.pkl','rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

In [ ]:
# Figure out if we have data or not for primaries
primary_has_data = []

for i in range(n_mw):

    # Get the primary
    primary = tree_primaries[i]
    n_snap = len(primary.snapnum)
    _primary_has_data = np.zeros(n_snap,dtype=bool)

    # Check for each snapshot
    for j in range(n_snap):
        sn = primary.snapnum[j]
        sid = primary.subfind_id[j]
        snap_path = data_dir+'cutouts/snap_'+str(sn)+'/'
        snap_filename = snap_path+'cutout_'+str(sid)+'.hdf5'
        _primary_has_data[j] = os.path.isfile(snap_filename)
    
    primary_has_data.append(_primary_has_data)

In [ ]:
# Now for major mergers
major_merger_has_data = []

for i in range(n_mw):

    # Get the primary
    primary = tree_primaries[i]
    n_major = primary.n_major_mergers
    _major_merger_has_data = []

    # Loop over major mergers
    for j in range(n_major):

        major_merger = primary.tree_major_mergers[j]
        n_snap = len(major_merger.snapnum)
        _major_merger_has_data_snap = np.zeros(n_snap,dtype=bool)

        # Check for each snapshot
        for k in range(n_snap):

            sn = major_merger.snapnum[k]
            sid = major_merger.subfind_id[k]
            snap_path = data_dir+'cutouts/snap_'+str(sn)+'/'
            snap_filename = snap_path+'cutout_'+str(sid)+'.hdf5'
            _major_merger_has_data_snap[k] = os.path.isfile(snap_filename)

        _major_merger_has_data.append(_major_merger_has_data_snap)
    major_merger_has_data.append(_major_merger_has_data)

### Do a super rough count of the number of halos for which we have / don't have data

In [ ]:
# Primaries
n_have = 0
n_missing = 0

for i in range(n_mw):

    # Get the primary
    primary = tree_primaries[i]
    n_snap = len(primary.snapnum)

    n_have = n_have + np.sum(primary_has_data[i])
    n_missing = n_missing + np.sum(~primary_has_data[i])

print(n_have,n_missing)
print(n_have+n_missing)
print(n_have/(n_have+n_missing))

In [ ]:
# Major mergers
n_have = 0
n_missing = 0

for i in range(n_mw):

    # Get the primary
    primary = tree_primaries[i]
    n_major = primary.n_major_mergers

    # Loop over major mergers
    for j in range(n_major):

        n_have = n_have + np.sum(major_merger_has_data[i][j])
        n_missing = n_missing + np.sum(~major_merger_has_data[i][j])

print(n_have,n_missing)
print(n_have+n_missing)
print(n_have/(n_have+n_missing))